In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from mosaicml.constants import MLModelFlavours
from werkzeug.datastructures import FileStorage
import pandas as pd
import os, tempfile

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# os.getenv("PROJECT_ID")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

# import time
# time.sleep(1000)
# print("-----creating spark session")
# spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
# spark.sparkContext.setLogLevel('WARN')

print("-----reading data from azure")
storage_account_name = "pocmosaic"
storage_account_access_key = "iX6jZejeNh1bUejdSRYk+4X2S/pAbTy+idkTM5UAw2HxgxSnaYGJSN+ffzZAQ5/Q6xQe2Ja74Re9nKHMVmsGhA=="
file_location = "wasbs://container1@pocmosaic.blob.core.windows.net/catalog/local_file_upload/RatanBBostonHousing.csv"
file_type = "csv"
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
dataset = spark.read.option("header", "true").format(file_type).option("inferSchema", "true").load(file_location)
dataset.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----reading data from azure
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631

In [2]:

print("-----splitting data")
train_data,test_data = dataset.randomSplit([0.8,0.2])

print("-----creating pipeline")
stage_1 = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
# define stage 5: logistic regression model                          
stage_2 = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

# setup the pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'dataset' is not defined
Traceback (most recent call last):
NameError: name 'dataset' is not defined



In [5]:
print("-----training pipeline")
regression_pipeline = regression_pipeline.fit(train_data)
test_update = regression_pipeline.transform(test_data)
# test_update.show()

#Predict using the model
print("-----test data")
test_update.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----training pipeline
-----test data
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|          Attributes|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01311|90.0| 1.22|   0| 0.403|7.249|21.9| 8.6966|  5|226|   17.9|395.93| 4.81|35.4|[0.01311,90.0,1.2...|31.205337419509313|
|0.02009|95.0| 2.68|   0|0.4161|8.034|31.9|  5.118|  4|224|   14.7|390.55| 2.88|50.0|[0.02009,95.0,2.6...|43.400883854339405|
|0.02187|60.0| 2.93|   0| 0.401|  6.8| 9.9| 6.2196|  1|265|   15.6|393.37| 5.03|31.1|[0.02187,60.0,2.9...| 31.87756003666385|
|0.03041| 0.0| 5.19|   0| 0.515|5.895|59.6|  5.615|  5|224|   20.2|394.81|10.56|18.5|[0.03041,0.0,5.19...|19.152514360359643|
|0.03548|80.0| 3.64|   0| 0.392|5.876|19.1| 9.2203|  1|315|   16.4|395.18| 9.25|

In [6]:

# define score function
def score(model, request):
    data = request.json
    df = pd.DataFrame(data, columns=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv'])
    spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
    spark_df = spark.createDataFrame(df)
    return model.transform(spark_df).toPandas()["prediction"].to_json()

print("-----defining & testing score function")
import requests
req = requests.Request()
req.json = [[1, -0.1, 0.0425, 0.0279, 2.4716, -15.2808, 4.2783, -0.0005, -1.4009, 0.2692, -0.013, -0.9905, 0.0084, -0.5533]]
rez = score(regression_pipeline, req)
print("the prediction is {}".format(rez))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----defining & testing score function
the prediction is {"0":-71.603733455}

In [7]:
%%local
import os
ai_server = os.environ["MOSAIC_AI_SERVER"]
project_id = os.environ["PROJECT_ID"]
token = os.environ["TOKEN"]

In [8]:
%%send_to_spark -i ai_server -t str -n ai_server

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'ai_server' as 'ai_server' to Spark kernel

In [9]:
%%send_to_spark -i project_id -t str -n project_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'project_id' as 'project_id' to Spark kernel

In [10]:
%%send_to_spark -i token -t str -n token

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'token' as 'token' to Spark kernel

In [11]:
import os
os.environ["MOSAIC_AI_SERVER"] = ai_server
os.environ["PROJECT_ID"] = project_id
os.environ["TOKEN"] = token

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
os.environ["MOSAIC_AI_SERVER"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'http://mosaic-ai-backend:5000/registry/api'

In [12]:
# def request_patched(method, url, **kwargs):
#     with requests.sessions.Session() as session:
#         url = url.replace("mosaic.dev.lti-mosaic.com", "lens.dev.lti-mosaic.com")
#         kwargs["verify"] = False
        
#         print("URL", url)
#         print("kwargs", kwargs)
#         result = session.request(method=method, url=url, **kwargs)
#         result.raise_for_status()
#         return result


# requests.api.request = request_patched

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
os.getenv("MOSAIC_AI_SERVER")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'http://mosaic-ai-backend:5000/registry/api'

In [14]:
import requests
url = "http://mosaic-ai-backend:5000/registry/api/ping"
# # # , headers={"X-Auth-Username":"vaibhav.bade@lntinfotech.com"})
resp = requests.get(url)
resp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [401]>

In [21]:
%%local import os 
os.getenv("MYSQL_PASSWORD")

''

In [16]:
from mosaicml import *
from mosaicml import constants
constants.MosaicAI.server = "http://mosaic-ai-backend:5000/registry/api"
constants.MosaicAI.server
# os.getenv("MOSAIC_AI_SERVER", "http://localhost:5000/registry/api")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'http://mosaic-ai-backend:5000/registry/api'

In [17]:
register_model(regression_pipeline, score, "Boston_Regression_Analysis_inter_july1", "pyspark model", MLModelFlavours.pyspark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------- pyspark
----------- True
----------- /spark-model-dir/31665726-db1d-11eb-9fb5-6a8587652774/ml_model

In [17]:
os.environ

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

environ({'EEF_KNOWLEDGE_MINING_METADATA_PORT_80_TCP_ADDR': '10.2.0.239', 'SVC_2F3BE5DE_6439_4387_9299_F0613CDD357B_PORT_5001_TCP_PORT': '5001', 'SVC_926DC36D_17C2_411E_9628_BC7D7FFB1D03_SERVICE_HOST': '10.2.1.30', 'MINIO_SERVICE_GCS_DEMO_PORT': 'tcp://10.2.0.243:9000', 'WORKFLOW_EXECUTION_1143793_1157889_PORT_80_TCP': 'tcp://10.2.1.35:80', 'WORKFLOW_EXECUTION_1146633_1160705_PORT': 'tcp://10.2.3.207:80', 'WORKFLOW_EXECUTION_1137137_1151290_SERVICE_HOST': '10.2.1.106', 'WORKFLOW_EXECUTION_1131539_1145734_PORT_80_TCP_PROTO': 'tcp', 'AIOPS_NOTIFICATION_SERVICE_PORT': 'tcp://10.2.2.210:80', 'JUPYTER_ENDPOINTS_D1442CC5_15A4_4E87_A007_89C1E513C336_PORT_80_TCP_ADDR': '10.2.0.65', 'WORKFLOW_EXECUTION_1126726_1140964_SERVICE_PORT': '80', 'WORKFLOW_EXECUTION_1145245_1159329_PORT_80_TCP': 'tcp://10.2.1.151:80', 'ENTITYEXTRACTOR_BACKEND_PORT_80_TCP': 'tcp://10.2.2.34:80', 'SVC_4BADA811_751E_4C79_879D_4F027911552F_PORT_5001_TCP_PROTO': 'tcp', 'WORKFLOW_EXECUTION_1131958_1146150_PORT_80_TCP': 'tcp:/